# 08 - Bias Field Correction with Dipý (N4)

This notebook demonstrates how to perform bias field correction using the N4 algorithm, which is available through Dipý and wrapped by the `diffusemri` library. 

**What is Bias Field?**
A bias field (also known as intensity non-uniformity or INU) is a low-frequency, spatially varying artifact that affects MR images. It causes the intensity of the same tissue type to vary across the image, which can negatively impact quantitative analysis, segmentation, and registration.

**N4 Bias Field Correction**
The N4 (N3 followed by B-spline fitting) algorithm is a popular method for correcting bias fields. It is an iterative method that models the bias field as a smooth multiplicative field. Dipý provides access to N4 implementations (often relying on ITK components, potentially via SimpleITK, or its own DeepN4 variant).

## Dependencies

This notebook and the wrapped functions rely on:
*   `diffusemri`: For the wrapper function.
*   `dipy`: Provides the core N4 bias field correction workflow.
*   `nibabel`: For NIfTI I/O.
*   `numpy`: For numerical operations.
*   `matplotlib`: For plotting.
*   Potentially `SimpleITK` or underlying ITK libraries: Dipý's N4 implementation may have dependencies on ITK, which might be satisfied if SimpleITK is installed or if Dipý was compiled with ITK support. Ensure your Dipý installation is complete with its optional dependencies for N4 if you encounter issues.

In [ ]:
import os
import shutil
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
# import SimpleITK as sitk # Not strictly needed for this notebook's dummy data creation with NumPy

# diffusemri library imports
from preprocessing.correction import correct_bias_field_dipy

# For conceptual CLI examples, one might import the main function:
# from cli.run_preprocessing import main as preprocessing_cli_main

# Setup a temporary directory for example files
TEMP_DIR = "temp_bias_correction_example"
if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
os.makedirs(TEMP_DIR)

print(f"Temporary directory for examples: {os.path.abspath(TEMP_DIR)}")

# Helper function for plotting slices
def show_slice(data_vol, slice_idx=None, title="", cmap='gray', vmin=None, vmax=None):
    """Displays a central slice of 2D, 3D, or 4D data (first volume for 4D)."""
    data_to_show = None
    if data_vol.ndim == 4:
        s_idx = slice_idx if slice_idx is not None else data_vol.shape[2] // 2
        data_to_show = data_vol[:, :, s_idx, 0] # Show first volume if 4D
    elif data_vol.ndim == 3:
        s_idx = slice_idx if slice_idx is not None else data_vol.shape[2] // 2
        data_to_show = data_vol[:, :, s_idx]
    elif data_vol.ndim == 2:
        data_to_show = data_vol # Already a 2D slice
    else:
        print(f"Cannot display data with {data_vol.ndim} dimensions.")
        return
    
    plt.figure(figsize=(6,5))
    plt.imshow(data_to_show.T, cmap=cmap, origin='lower', vmin=vmin, vmax=vmax)
    plt.title(title)
    plt.xlabel("X voxel index"); plt.ylabel("Y voxel index")
    plt.colorbar(label="Intensity")
    plt.show()

## Part 1: Creating Dummy Data with a Simulated Bias Field

To demonstrate bias field correction, we first need an image that has a bias field. We'll create a simple 3D NIfTI image and then apply a synthetic multiplicative bias field to it.

In [ ]:
# Create a base dummy 3D NIfTI image
dims = (64, 64, 20)  # X, Y, Z dimensions
base_data = np.zeros(dims, dtype=np.float32)

# Create a central rectangular region with higher signal
base_data[20:44, 20:44, 5:15] = 200.0
# Add some random noise
base_data += np.random.normal(loc=0, scale=10, size=dims)
base_data = np.clip(base_data, 0, None) # Ensure no negative values from noise

affine = np.diag([2.0, 2.0, 2.0, 1.0])  # Simple 2mm isotropic voxel affine
original_nifti_path = os.path.join(TEMP_DIR, "original_image.nii.gz")
nib.save(nib.Nifti1Image(base_data, affine), original_nifti_path)

print(f"Created original dummy NIfTI: {original_nifti_path}")
show_slice(base_data, title="Original Data (Slice in Z)")

In [ ]:
# Simulate a simple multiplicative bias field
# Create coordinate grids
x_coords = np.linspace(-1, 1, dims[0])
y_coords = np.linspace(-1, 1, dims[1])
z_coords = np.linspace(-1, 1, dims[2])
xx, yy, zz = np.meshgrid(x_coords, y_coords, z_coords, indexing='ij')

# Define a bias field: e.g., brighter towards one corner (positive x, y, z)
# and with a slight quadratic falloff from the center of x,y plane
bias_field = 0.8 + 0.5 * (xx + yy + zz) - 0.3 * (xx**2 + yy**2) 

# Normalize the field to be positive and have a reasonable range (e.g., 0.6 to 1.4)
bias_field_min = bias_field.min()
bias_field_max = bias_field.max()
bias_field_normalized = (bias_field - bias_field_min) / (bias_field_max - bias_field_min) # Range 0-1
bias_field_scaled = 0.6 + 0.8 * bias_field_normalized # Example: range 0.6 to 1.4

# Apply the bias field multiplicatively
biased_data = base_data * bias_field_scaled
biased_data = np.clip(biased_data, 0, None) # Ensure positivity

biased_nifti_path = os.path.join(TEMP_DIR, "biased_image.nii.gz")
nib.save(nib.Nifti1Image(biased_data, affine), biased_nifti_path)

print(f"Created NIfTI with simulated bias field: {biased_nifti_path}")
# Show the biased data, using vmin/vmax from original for comparison if desired
show_slice(biased_data, title="Data with Simulated Bias Field", vmin=base_data.min(), vmax=base_data.max()*1.4) 
show_slice(bias_field_scaled, title="Simulated Bias Field Multiplier", cmap='viridis')

## Part 2: Applying N4 Bias Field Correction

Now we use the `correct_bias_field_dipy` function from `diffusemri.preprocessing.correction` to apply N4 correction to our biased image. This function wraps Dipý's N4 implementation.

In [ ]:
corrected_nifti_path = os.path.join(TEMP_DIR, "n4_corrected_image.nii.gz")
estimated_bias_field_by_n4_path = None # The current wrapper does not return this path directly

print(f"Applying N4 bias field correction to: {biased_nifti_path}")
print(f"Corrected image will be saved to: {corrected_nifti_path}")

try:
    # The N4 algorithm in Dipý typically works on 3D images. 
    # If you have a 4D DWI series, you might apply it to a mean b0 image, 
    # or iterate through volumes if appropriate (though N4 is usually for T1w/T2w or mean DWI).
    # Our dummy data `biased_nifti_path` is 3D.
    
    # The wrapper function might take additional kwargs for N4 if exposed.
    # Common N4 parameters include: shrink_factor, n_iterations, convergence_threshold.
    # For dummy data, default or simplified parameters might be needed for quick execution.
    # Example of potential kwargs (if the wrapper supports them, check its docstring):
    # n4_kwargs = {'n_iterations': [50,40,30], 'convergence_threshold': 1e-4}

    output_path_n4 = correct_bias_field_dipy(
        input_image_file=biased_nifti_path,
        output_corrected_file=corrected_nifti_path,
        method='n4' # Explicitly request 'n4'
        # **n4_kwargs # Pass additional N4 parameters here if supported
    )
    
    if os.path.exists(output_path_n4):
        print(f"N4 bias field correction completed. Corrected image saved to: {output_path_n4}")
        corrected_data_n4 = nib.load(output_path_n4).get_fdata()
        show_slice(corrected_data_n4, title="N4 Corrected Data", vmin=base_data.min(), vmax=base_data.max()*1.4)
        
        # The current `correct_bias_field_dipy` wrapper does not directly return the estimated bias field.
        # However, Dipý's N4BiasFieldCorrection workflow internally calculates it. 
        # If it were saved by the workflow (e.g. if the nipype interface does this),
        # one could try to load it. For now, we just show the corrected image.
        # Example: if a bias field file was saved as `*_bias_field.nii.gz` by the underlying tool:
        # potential_bias_field_path = corrected_nifti_path.replace(".nii.gz", "_bias_field.nii.gz")
        # if os.path.exists(potential_bias_field_path):
        #    estimated_bias_field_data = nib.load(potential_bias_field_path).get_fdata()
        #    show_slice(estimated_bias_field_data, title="Estimated Bias Field by N4", cmap='viridis')
        # else:
        #    print("Estimated bias field file not found at expected conventional path.")
            
    else:
        print("N4 correction function ran, but the output file was not found.")

except Exception as e:
    print(f"An error occurred during N4 bias field correction: {e}")
    print("This could be due to the synthetic nature of the dummy data, missing ITK dependencies for Dipý's N4, or N4 algorithm parameters not being robust for this simple data.")

### CLI Conceptual Example

Bias field correction can also be invoked via the `run_preprocessing.py` script:

```bash
# Conceptual CLI command - replace paths with your actual file paths
python cli/run_preprocessing.py bias_field_dipy \
  --input_file /path/to/your/image_with_bias.nii.gz \
  --output_file /path/to/output/corrected_image.nii.gz \
  --method n4 
  # Optional: --mask_file /path/to/your/brain_mask.nii.gz (N4 often performs better with a mask)
  # Optional: --n4_iterations 50 40 30 (example if CLI supports passing N4 specific args)
```

## Discussion

*   **Effectiveness**: The N4 algorithm is generally effective, but its performance can depend on the characteristics of the bias field, the image data itself, and the chosen parameters.
*   **Parameters**: Dipý's N4 implementation (and ITK's N4) has several parameters that can be tuned, such as:
    *   `n_iterations`: Number of iterations at each level of a multi-resolution pyramid (e.g., `[50,40,30,20]`).
    *   `convergence_threshold`: Criterion to stop iterations.
    *   `shrink_factor`: Downsampling factor at each level of the pyramid.
    *   `spline_param` or `bspline_fitting_distance`: Controls the smoothness of the estimated bias field.
    The `diffusemri` wrapper `correct_bias_field_dipy` may expose some of these via `**kwargs`. Refer to its documentation or Dipý's documentation for details.
*   **Masking**: Applying N4 within a brain mask can sometimes improve results by focusing the algorithm on relevant tissues and avoiding influence from noisy background or skull regions.
*   **Input Data**: While N4 is often applied to T1-weighted anatomical images, it can also be applied to other MR modalities, including mean b0 images from a DWI series if they suffer from significant bias field.

## Cleanup

Remove the temporary directory and its contents.

In [ ]:
if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
    print(f"Cleaned up temporary directory: {TEMP_DIR}")